In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Define paramaters for the model

In [3]:
learning_rate = 0.01
batch_size = 128
n_epochs = 10

### Step 1: Read in data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Step 2: create placeholders for features and labels
Each image in the MNIST data is of shape 28 by 28 = 784. Therefore, each image is represented with a 1x784 tensor.        
There are 10 classes for each image, corresponding to digits 0 - 9.     
Features are of the type float, and labels are of the type int.  

In [5]:
X = tf.placeholder(tf.float32, [None, 784], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")

### Step 3: create weights and bias
weights and biases are initialized to 0   
shape of w depends on the dimension of X and Y so that Y = X * w + b   
shape of b depends on Y   

In [6]:
W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='W')
b = tf.Variable(tf.zeros([1, 10]), name="b")

### Step 4: build model
The model that returns the logits. This logits will be later passed through softmax layer to get the probability distribution of possible label of the image

In [7]:
logits = tf.add(tf.matmul(X, W), b)

# Step 5: define loss function
Use cross entropy loss of the real labels with the softmax of logits

In [8]:
entropy  = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y, name = 'loss')
loss = tf.reduce_mean(entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Step 6: define training op

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [10]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X:X_batch, Y:Y_batch})
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(

    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += sum(accuracy_batch)

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.3691047687769492
Average loss epoch 1: 0.29305654378620893
Average loss epoch 2: 0.2818339346693112
Average loss epoch 3: 0.2806101925584264
Average loss epoch 4: 0.2758264170660006
Average loss epoch 5: 0.27302348582477837
Average loss epoch 6: 0.2694404195551272
Average loss epoch 7: 0.2662625656504453
Average loss epoch 8: 0.26656612713923267
Average loss epoch 9: 0.2657758221724928
Total time: 6.559999942779541 seconds
Optimization Finished!
Accuracy 0.9202
